In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
import os
from pathlib import Path
import datetime
import csv

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
epoch = datetime.datetime.utcfromtimestamp(0)

def unix_time_secs(dt):
    return round((dt - epoch).total_seconds())

def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def nn_example(e, b, data_type):
    learning_rate = 0.5
    epochs = e
    batch_size = b
    input_feature_count = 784
    out_classes = 10
    data_type = data_type
    
    # Neural network hidden layer variables
    h1 = 500
    h2 = 200

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(data_type, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(data_type, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(data_type)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], data_type)
    b_fc1 = bias_variable([h1], data_type)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(data_type)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],data_type)
    b_fc2 = bias_variable([h2], data_type)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],data_type)
    b_fc3 = bias_variable([out_classes], data_type)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, data_type))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)
    
    saver = tf.train.Saver()

    test_accuracy = 0
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        start_time = time.time()
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
            elapsed = time.time() - start_time
            test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#             print((epoch+1), batch_size, "{:.3f}".format(avg_cost), "{:.3f}".format(test_accuracy), "{:.3f}".format(elapsed), sep='\t')
#     return arr_accuracy, time_elapsed, nw_size

if __name__ == "__main__":
    data_type = tf.float64
    filename = r'D:\UCI\HPA_Project\GPU-Z Data\f64.csv'
    file = open(filename, "w", newline='\n')
    batch_size = [500, 1000, 2000, 5000, 10000, 20000]
    epochs = 15
    num_bsize = len(batch_size)
    mat = np.zeros((num_bsize, 4))
    start_time = 0
    for i, b in enumerate(batch_size):
        print('Training for b={}'.format(b), end=' ')
        start_time = datetime.datetime.now()
        nn_example(epochs, b, data_type)
        end_time = datetime.datetime.now()
        line = str(b)+','+str(unix_time_secs(start_time))+','+str(unix_time_secs(end_time))+','+str(unix_time_secs(end_time)-unix_time_secs(start_time))
        file.write(line)
        print('Duration={}'.format(unix_time_secs(end_time)-unix_time_secs(start_time)))
    file.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Training for b=500 Duration=209
Training for b=1000 Duration=164
Training for b=2000 Duration=165
Training for b=5000 Duration=160
Training for b=10000 Duration=148
Training for b=20000 Duration=126


In [ ]:
if __name__ == "__main__":
    data_type = tf.float32
    filename = r'D:\UCI\HPA_Project\GPU-Z Data\f32.csv'
    file = open(filename, "w", newline='\n')
    batch_size = [500, 1000, 2000, 5000, 10000, 20000]
    epochs = 15
    num_bsize = len(batch_size)
    mat = np.zeros((num_bsize, 4))
    start_time = 0
    for i, b in enumerate(batch_size):
        print('Training for b={}'.format(b), end=' ')
        start_time = datetime.datetime.now()
        nn_example(epochs, b, data_type)
        end_time = datetime.datetime.now()
        line = str(b)+','+str(unix_time_secs(start_time))+','+str(unix_time_secs(end_time))+','+str(unix_time_secs(end_time)-unix_time_secs(start_time))
        file.write(line)
        print('Duration={}'.format(unix_time_secs(end_time)-unix_time_secs(start_time)))
    file.close()

Training for b=500 Duration=49
Training for b=1000 Duration=47
Training for b=2000 Duration=43
Training for b=5000 Duration=41
Training for b=10000 Duration=43
Training for b=20000 Duration=38


In [ ]:
if __name__ == "__main__":
    data_type = tf.float16
    filename = r'D:\UCI\HPA_Project\GPU-Z Data\f16.csv'
    file = open(filename, "w", newline='\n')
    batch_size = [500, 1000, 2000, 5000, 10000, 20000]
    epochs = 15
    num_bsize = len(batch_size)
    mat = np.zeros((num_bsize, 4))
    start_time = 0
    for i, b in enumerate(batch_size):
        print('Training for b={}'.format(b), end=' ')
        start_time = datetime.datetime.now()
        nn_example(epochs, b, data_type)
        end_time = datetime.datetime.now()
        line = str(b)+','+str(unix_time_secs(start_time))+','+str(unix_time_secs(end_time))+','+str(unix_time_secs(end_time)-unix_time_secs(start_time))
        file.write(line)
        print('Duration={}'.format(unix_time_secs(end_time)-unix_time_secs(start_time)))
    file.close()

Training for b=500 Duration=45
Training for b=1000 Duration=44
Training for b=2000 

In [17]:
date='20171201'
time='18:08:00'
yyyy, MM, dd = date[:4], date[4:6], date[6:]
hh, mm, ss = time.split(':')
print(yyyy, MM, dd)
print(hh, mm, ss)
print(datetime.datetime(int(yyyy), int(MM), int(dd), int(hh), int(mm), int(ss)))
print(unix_time_secs(datetime.datetime(int(yyyy), int(MM), int(dd), int(hh), int(mm), int(ss))))
print(unix_time_secs(datetime.datetime.now()))

2017 12 01
18 08 00
2017-12-01 18:08:00
1512151680
1512151680


In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
import os
from pathlib import Path
import datetime

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def nn_example(e, b, data_type):
    learning_rate = 0.5
    epochs = e
    batch_size = b
    input_feature_count = 784
    out_classes = 10
    data_type = data_type
    
    # Neural network hidden layer variables
    h1 = 500
    h2 = 200

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(data_type, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(data_type, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(data_type)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], data_type)
    b_fc1 = bias_variable([h1], data_type)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(data_type)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],data_type)
    b_fc2 = bias_variable([h2], data_type)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],data_type)
    b_fc3 = bias_variable([out_classes], data_type)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, data_type))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)
    
    saver = tf.train.Saver()

    test_accuracy = 0
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        start_time = time.time()
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
            elapsed = time.time() - start_time
            test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#             print((epoch+1), batch_size, "{:.3f}".format(avg_cost), "{:.3f}".format(test_accuracy), "{:.3f}".format(elapsed), sep='\t')
#     return arr_accuracy, time_elapsed, nw_size

if __name__ == "__main__":
    data_type = tf.float32
    filename = r'D:\UCI\HPA_Project\GPU-Z Data\f32.csv'
    file = open(filename, "w", newline='\n')
    batch_size = [50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 30000, 40000, 50000, 55000]
    epochs = 15
    num_bsize = len(batch_size)
    mat = np.zeros((num_bsize, 4))
    start_time = 0
    for i, b in enumerate(batch_size):
        print('Training for b={}'.format(b))
        start_time = datetime.datetime.now()
        nn_example(epochs, b, data_type)
        end_time = datetime.datetime.now()
        line = str(b)+','+str(unix_time_secs(start_time))+','+str(unix_time_secs(end_time))+','+str(unix_time_secs(end_time)-unix_time_secs(start_time))
        file.write(line)
    file.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Training for b=50
Training for b=100
Training for b=200
Training for b=500
Training for b=1000
Training for b=2000
Training for b=5000
Training for b=10000
Training for b=20000
Training for b=30000
Training for b=40000
Training for b=50000
Training for b=55000


In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
import os
from pathlib import Path
import datetime

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def nn_example(e, b, data_type):
    learning_rate = 0.5
    epochs = e
    batch_size = b
    input_feature_count = 784
    out_classes = 10
    data_type = data_type
    
    # Neural network hidden layer variables
    h1 = 500
    h2 = 200

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(data_type, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(data_type, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(data_type)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], data_type)
    b_fc1 = bias_variable([h1], data_type)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(data_type)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],data_type)
    b_fc2 = bias_variable([h2], data_type)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],data_type)
    b_fc3 = bias_variable([out_classes], data_type)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, data_type))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)
    
    saver = tf.train.Saver()

    test_accuracy = 0
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        start_time = time.time()
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
            elapsed = time.time() - start_time
            test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
            print((epoch+1), batch_size, "{:.3f}".format(avg_cost), "{:.3f}".format(test_accuracy), "{:.3f}".format(elapsed), sep='\t')
#     return arr_accuracy, time_elapsed, nw_size

if __name__ == "__main__":
    data_type = tf.float16
    filename = r'D:\UCI\HPA_Project\GPU-Z Data\f16.csv'
    file = open(filename, "w", newline='\n')
    batch_size = [50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 30000]
    epochs = 15
    num_bsize = len(batch_size)
    mat = np.zeros((num_bsize, 4))
    start_time = 0
    for i, b in enumerate(batch_size):
        print('Training for b={}'.format(b))
        start_time = datetime.datetime.now()
        nn_example(epochs, b, data_type)
        end_time = datetime.datetime.now()
        line = str(b)+','+str(unix_time_secs(start_time))+','+str(unix_time_secs(end_time))+','+str(unix_time_secs(end_time)-unix_time_secs(start_time))+'\n'
        file.write(line)
    file.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Training for b=50
1	50	nan	0.098	11.641
2	50	nan	0.098	23.466
3	50	nan	0.098	34.487
4	50	nan	0.098	45.866
5	50	nan	0.098	56.040
6	50	nan	0.098	66.232
7	50	nan	0.098	80.117
8	50	nan	0.098	90.618
9	50	nan	0.098	101.140
10	50	nan	0.098	111.472
11	50	nan	0.098	121.899
12	50	nan	0.098	132.334
13	50	nan	0.098	143.207
14	50	nan	0.098	160.933
15	50	nan	0.098	174.820
Training for b=100
1	100	nan	0.098	4.480
2	100	nan	0.098	8.662
3	100	nan	0.098	12.637
4	100	nan	0.098	16.534
5	100	nan	0.098	20.525
6	100	nan	0.098	24.496
7	100	nan	0.098	28.442
8	100	nan	0.098	32.403
9	100	nan	0.098	36.335
10	100	nan	0.098	40.317
11	100	nan	0.098	44.772
12	100	nan	0.098	50.616
13	100	nan	0.098	55.687
14	100	nan	0.098	59.765
15	100	nan	0.098	63.736
Training for b=200
1	200	nan	0.098	2.436
2	200	nan	0.098	4.969
3	200	nan	0.09

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
import os
from pathlib import Path

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def nn_example(e, b, data_type):
    learning_rate = 0.5
    epochs = e
    batch_size = b
    input_feature_count = 784
    out_classes = 10
    data_type = data_type
    max_epoch = epochs[-1]
    
    # Neural network hidden layer variables
    h1 = 50
    h2 = 20

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(data_type, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(data_type, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(data_type)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], data_type)
    b_fc1 = bias_variable([h1], data_type)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(data_type)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],data_type)
    b_fc2 = bias_variable([h2], data_type)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],data_type)
    b_fc3 = bias_variable([out_classes], data_type)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, data_type))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)
    
    saver = tf.train.Saver()

    merged = tf.summary.merge([accuracy_sum])
    writer = tf.summary.FileWriter(r'C:\Users\anant\dev\repos\HPA')
    
    test_accuracy = 0
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        start_time = time.time()
        for epoch in range(max_epoch+1):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
            if(epoch in epochs):
                elapsed = time.time() - start_time
#                 filepath = "D:\\UCI\\HPA_Project\\weights_data\\epoch_"+ str(epoch)+ '_batch_' +str(batch_size)+'_'+str(data_type)
#                 save_path = saver.save(sess, filepath)
#                 mfilepath = filepath + ".data-00000-of-00001"
#                 file = Path(mfilepath)
#                 size = file.stat().st_size
                test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
                print(epoch, batch_size, test_accuracy, elapsed)
#     return arr_accuracy, time_elapsed, nw_size

if __name__ == "__main__":
    data_type = tf.float32
    batch_size = [50, 100, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 400, 500]
#     epochs = [5,10, 15, 20] #, 30, 50] #, 80, 100, 150, 200, 300, 500, 700, 1000]
    epochs = [15]
    num_epochs = len(epochs)
    num_bsize = len(batch_size)
    for j, b in enumerate(batch_size):
        nn_example(epochs, b, data_type)
        print()

import matplotlib as plt
plt.plot()

accuracy_mat_64 = accuracy_mat
elapsed_time_64 = elapsed_time
size_mat_64 = size_mat

accuracy_mat_32 = accuracy_mat
elapsed_time_32 = elapsed_time
size_mat_32 = size_mat

print(accuracy_mat)